In [12]:
import os
import json
from typing import List

from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_community.graphs import Neo4jGraph


os.environ["OPENAI_API_KEY"] = "sk-"

llm = ChatOpenAI(model_name="gpt-4-turbo", request_timeout=180)

In [1]:
query_types = {
    "Simple Retrieval Queries": "These queries focus on basic data extraction, retrieving nodes or relationships based on straightforward criteria such as labels, properties, or direct relationships. Examples include fetching all nodes labeled as 'Person' or retrieving relationships of a specific type like 'EMPLOYED_BY'. Simple retrieval is essential for initial data inspections and basic reporting tasks. Always limit the number of results if more than one row is expected from the questions by saying 'first 3' or 'top 5' elements",
    "Complex Retrieval Queries": "These advanced queries use the rich pattern-matching capabilities of Cypher to handle multiple node types and relationship patterns. They involve sophisticated filtering conditions and logical operations to extract nuanced insights from interconnected data points. An example could be finding all 'Person' nodes who work in a 'Department' with over 50 employees and have at least one 'REPORTS_TO' relationship. Always limit the number of results if more than one row is expected from the questions by saying 'first 3' or 'top 5' elements",
    "Simple Aggregation Queries": "Simple aggregation involves calculating basic statistical metrics over properties of nodes or relationships, such as counting the number of nodes, averaging property values, or determining maximum and minimum values. These queries summarize data characteristics and support quick analytical conclusions. Always limit the number of results if more than one row is expected from the questions by saying 'first 3' or 'top 5' elements",
    "Pathfinding Queries": "Specialized in exploring connections between nodes, these queries are used to find the shortest path, identify all paths up to a certain length, or explore possible routes within a network. They are essential for applications in network analysis, routing, logistics, and social network exploration. Always limit the number of results if more than one row is expected from the questions by saying 'first 3' or 'top 5' elements",
    "Complex Aggregation Queries": "The most sophisticated category, these queries involve multiple aggregation functions and often group results over complex subgraphs. They calculate metrics like average number of reports per manager or total sales volume through a network, supporting strategic decision making and advanced reporting. Always limit the number of results if more than one row is expected from the questions by saying 'first 3' or 'top 5' elements",
    "Verbose query": "These queries are characterized by their explicit and detailed specifications about the data retrieval process and the exact information needed. They involve elaborate instructions for navigating through complex data structures, specifying precise criteria for inclusion, exclusion, and sorting of data points. Verbose queries typically require the breakdown of each step in the querying process, from the initial identification of relevant data nodes and relationships to the intricate filtering and sorting mechanisms that must be applied. Always limit the number of results if more than one row is expected from the questions by saying 'first 3' or 'top 5' elements",
}

In [13]:
class Question(BaseModel):
    questions: List[str] = Field(
        description="List of relevant questions for the particular graph schema. Make sure that questions can be answered with information from the schema and that the questions are diverse as possible. Make sure that the schema and the example values contains the information that can answer the questions! Do not ask questions that cannot be answered based on the provided schema. For example, if no information about subtitles can be found in the graph, don't ask any information about subtitles. Make sure to always limit the results to less than 10 results by saying 3 users, or top 5 movies, or similar."
    )


structured_llm = llm.with_structured_output(Question)

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import (
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)


system_prompt = """Your task is to generate 100 questions that are directly related to a specific graph schema in Neo4j. Each question should target distinct aspects of the schema, such as relationships between nodes, properties of nodes, or characteristics of node types. Ensure that the questions vary in complexity, covering basic, intermediate, and advanced queries.

Avoid ambiguous questions. For clarity, an ambiguous question is one that can be interpreted in multiple ways or does not have a straightforward answer based on the schema. For example, avoid asking, "What is related to this?" without specifying the node type or relationship.
Please design each question to yield a limited number of results, specifically between 3 to 10 results. This will ensure that the queries are precise and suitable for detailed analysis and training.
The goal of these questions is to create a dataset for training AI models to convert natural language queries into Cypher queries effectively.
It is vital that the database contains information that can answer the question!
Make sure to generate 100 questions!"""

default_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            f"{system_prompt} Follow these instructions: {{instructions}}"
        ),
        HumanMessagePromptTemplate.from_template(
            "Make sure to create questions for the following graph schema:{input}\n Here are some example nodes and relationship values: {values}. Don't use any values that aren't found in the schema or in provided values. Also, do not ask questions that there is no way to answer based on the schema or provided example values. Don't include question index or the sequence of the question in the list."
        ),
    ]
)

chain = default_prompt | structured_llm

In [48]:
DEMO_URL = "neo4j+s://demo.neo4jlabs.com"
DEMO_DATABASES = [
    "recommendations",
    "buzzoverflow",
    "bluesky",
    "companies",
    "fincen",
    "gameofthrones",
    "grandstack",
    "movies",
    "neoflix",
    "network",
    "northwind",
    "offshoreleaks",
    "slack",
    "stackoverflow2",
    "twitch",
    "twitter",
];

In [ ]:
all_questions = []
for database in DEMO_DATABASES:
    print(database)
    graph = Neo4jGraph(
        url=DEMO_URL,
        database=database,
        username=database,
        password=database,
        enhanced_schema=True,
        sanitize=True,
    )
    schema = graph.schema
    for type in query_types:
        print(type)
        instructions = f"{type}: {query_types[type]}"
        values = graph.query(
            """MATCH (n)
WHERE rand() > 0.6
WITH n LIMIT 2
CALL {
    WITH n
    MATCH p=(n)-[*3..3]-()
    RETURN p LIMIT 1
}
RETURN p"""
        )
        questions = chain.invoke(
            {"input": schema, "instructions": instructions, "values": values}
        )
        all_questions.extend(
            [
                {"question": el, "type": type, "database": database}
                for el in questions.questions
            ]
        )
        # Get values for each type separately

In [39]:
len(all_questions)

9824

In [40]:
import random

# Get 10 random elements from the list
random_elements = random.sample(all_questions, 10)

print(random_elements)

[{'question': 'Identify the 3 answers with the lowest scores provided to the most viewed questions.', 'type': 'Simple Aggregation Queries', 'database': 'stackoverflow2'}, {'question': "List the first 3 countries where the filings have originated from entities based in 'Angola'.", 'type': 'Simple Retrieval Queries', 'database': 'fincen'}, {'question': "List the top 5 users having the most interactions with a specific key in 'INTERACTED' relationship.", 'type': 'Complex Retrieval Queries', 'database': 'bluesky'}, {'question': 'What are the user ids of those who have asked a question with a favorites count of zero?', 'type': 'Simple Retrieval Queries', 'database': 'buzzoverflow'}, {'question': 'Which 3 products have the lowest unit price and are still in stock?', 'type': 'Simple Retrieval Queries', 'database': 'northwind'}, {'question': "List all questions tagged with 'node.js' and asked by users with a reputation below 5000.", 'type': 'Simple Aggregation Queries', 'database': 'buzzoverfl

In [41]:
import pandas as pd

df = pd.DataFrame.from_records(all_questions)

In [42]:
df.groupby("database").size()

database
bluesky            135
buzzoverflow       627
companies          991
fincen             617
gameofthrones      399
grandstack         828
movies             767
neoflix            937
network            625
northwind          818
offshoreleaks      514
recommendations    792
slack              356
stackoverflow2     313
twitch             585
twitter            520
dtype: int64

In [43]:
df.to_csv("text2cypher_questions.csv", index=False)

In [49]:
schemas = {}
all_schemas = []
for database in DEMO_DATABASES:
    graph = Neo4jGraph(
        url=DEMO_URL,
        database=database,
        username=database,
        password=database,
        enhanced_schema=True,
        sanitize=True,
    )
    schema = graph.schema
    schemas[database] = schema
    all_schemas.append(
        {
            "database": database,
            "schema": schema,
            "structured_schema": graph.structured_schema,
        }
    )

df_schemas = pd.DataFrame.from_records(all_schemas)
df_schemas.to_csv("text2cypher_schemas.csv", index=False)

In [50]:
# Create OpenAI Batch processing JSONL file

system = """Given an input question, convert it to a Cypher query. No pre-amble.
Additional instructions:
- Ensure that queries checking for non-null properties use `IS NOT NULL` in a straightforward manner.
- Don't use `size((n)--(m))` for counting relationships. Instead use the new `count{(n)--(m))}` syntax.
- Incorporate the new existential subqueries in examples where the query needs to check for the existence of a pattern.
  Example: MATCH (p:Person)-[r:IS_FRIENDS_WITH]->(friend:Person)
            WHERE exists{ (p)-[:WORKS_FOR]->(:Company {name: 'Neo4j'})}
            RETURN p, r, friend"""


def create_line(question, database, id):
    schema = schemas[database]
    user_message = f"""Based on the Neo4j graph schema below, write a Cypher query that would answer the user's question:
    {schema}
    
    Question: {question}
    Cypher query:"""

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": user_message},
    ]
    line = {
        "custom_id": "request-" + str(id),
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-4-turbo",
            "messages": messages,
            "max_tokens": 1000,
            "temperature": 0,
        },
    }
    return line


with open("text2cypher_batch_input.jsonl", "w") as outfile:
    for i, q in enumerate(all_questions):
        line = create_line(q["question"], q["database"], i)
        json.dump(line, outfile)
        outfile.write("\n")